In [2]:
import pandas as pd
import numpy as np
from data import train_data_generator

# HMS vision
This project is aimed at building a machine learning model that detects smoke plumes in GOES satellite imagery in the same way as HMS does.
## Data
Data required for HMS vision is stored as png files on Mickley lab storage. 
### Data Description
#### Input Variables
HMS is produced from GOES Geostationary Satellite imagery. GOES provides two bandwidths, visible and infrared, both of which are used for HMS production. We downloaded GOES imagery for every thirty minutes from the beginning of 2017 to the end of 2021, and crop them from -180E to 0 in longitude, and from 90N to 0 in latitude. The resulting images have dimension of 12384x6192. For our traininnng purposes, we split them up into square tiles. The side length of the tiles have to be $2^n$ where $n\in\mathbb{N}$. For each tile, we examine if the tile is valid for training; we use tiles that does not contain void pixel (since GOES imagery was converted to rectangle image there are some void pixels on the edges) and is not on the night side.
#### Target Variable
Similarly to GOES imagery, we convert HMS into 12384x6192 images and split up into tiles. Each pixel of HMS is classified into four categories (0, 5, 16, and 27) depending on the concentration of PM2.5. We convert those categories into simple integer (0,1,2, and 3) so that the target variable is 2-D array where each element indicate the density level of smoke plume (no smoke, low, medium, and high).

### Implementation
#### Data Storage
All of GOES are stored at `/n/mickley/users/ktoshima/` with directory name `goes16_<band>_<year>` (i.e. `goes16_b1_2018` contains GOES band1 imagery from 2018). HMS imagery are stored at `/n/mickley/users/HMS/`. Since image storage is somewhat cluttered, there are csv files that organizes both GOES and HMS imagery. `/n/mickley/users/ktoshima/csv/path_all.csv` is a csv file contains paths for all GOES and HMS imagery organized by timestamps of the images. You can load it by

In [3]:
df = pd.read_csv('/n/mickley/users/ktoshima/csv/path_all.csv', index_col=['timestamp'], parse_dates=['timestamp'])
df.head()

,path_band1,path_band3,path_hms,path_daynight,split_256
timestamp,,,,,
2017-07-11 00:00:00,/n/mickley/users/ktoshima/goes16_b1_2017/GOES1...,/n/mickley/users/ktoshima/goes16_b3_2017/GOES1...,/n/mickley/users/HMS/HMS_Density/2017/HMS_Dens...,/n/mickley/users/HMS/DayNight/DayNight_192_000...,True
2017-07-11 00:30:00,/n/mickley/users/ktoshima/goes16_b1_2017/GOES1...,/n/mickley/users/ktoshima/goes16_b3_2017/GOES1...,/n/mickley/users/HMS/HMS_Density/2017/HMS_Dens...,/n/mickley/users/HMS/DayNight/DayNight_192_003...,True
2017-07-11 01:00:00,/n/mickley/users/ktoshima/goes16_b1_2017/GOES1...,/n/mickley/users/ktoshima/goes16_b3_2017/GOES1...,/n/mickley/users/HMS/HMS_Density/2017/HMS_Dens...,/n/mickley/users/HMS/DayNight/DayNight_192_010...,True
2017-07-11 01:30:00,/n/mickley/users/ktoshima/goes16_b1_2017/GOES1...,/n/mickley/users/ktoshima/goes16_b3_2017/GOES1...,/n/mickley/users/HMS/HMS_Density/2017/HMS_Dens...,/n/mickley/users/HMS/DayNight/DayNight_192_013...,True
2017-07-11 02:00:00,/n/mickley/users/ktoshima/goes16_b1_2017/GOES1...,/n/mickley/users/ktoshima/goes16_b3_2017/GOES1...,/n/mickley/users/HMS/HMS_Density/2017/HMS_Dens...,/n/mickley/users/HMS/DayNight/DayNight_192_020...,True


For each timestamp, paths for GOES (band1, band3), HMS, and DayNight are stored. For example,

In [4]:
df.loc['2017-07-15 16:00:00']

path_band1       /n/mickley/users/ktoshima/goes16_b1_2017/GOES1...
path_band3       /n/mickley/users/ktoshima/goes16_b3_2017/GOES1...
path_hms         /n/mickley/users/HMS/HMS_Density/2017/HMS_Dens...
path_daynight    /n/mickley/users/HMS/DayNight/DayNight_196_160...
split_256                                                     True
Name: 2017-07-15 16:00:00, dtype: object

#### Producing tiles
Producing square tiles from original imagery is done with two python files; `split_image.py` and `produce_tiles.py`. `split_image.py` contains necessart helper function to produce tiles, including image splitting and tile validation. Ultimately, you only need to use `split_image.split_image` method. `split_image` method takes in the side length of the tile, path for the set of imagery (GOES, HMS, DayNight), path to the directory you want to store your tiles, basename for the tiles, and timestamp. `produce_tiles.py` contains the codes I used to produce $256\times256$ tiles. When you would like to produce tiles, use `produce_tiles.py` as following;
```shell
python produce_tiles.py --side-len=256 --start-over=true
```
The code above will produce $256\times256$ tiles. `path_all.csv` contains a column that denotes if the splitting is done for each image. If `--start-over=false`, based on `path_all.csv` data, `produce_tiles.py` will split images that have not been splitted. If you want to start over the splitting you should use `--start-over=true`.

## Model
### Network architecture
For our model, we use U-net architecture; U-net is a deep learning architecture developed by Olaf Ronneberger et al. for biomedical image segmentation. The original model takes cell images as input and classifies if each pixel is on cell border or within a cell. In our case, the model takes in seven images over certain region at certain timestamp, and classifies each pixel in the input domain into one of four category of smoke density using HMS as target variable. The model first "encodes" the input image by applying $3\times3$ convolution layers with ReLU and $2\times2$ max pooling operations, and "decode" by applying upsampling convolution layers to compute pixel classification of the input from the feature map. We modify the original model in that we add padding to each convolution layers to preserve the length of the edge (the output image is shrinked in the original U-net).

## Training
### Preprocessing
We filter the dataset by how much smoke is in the target HMS image. Since most of the images have no smoke, when we train the model on the whole dataset, the model learns to always output "no smoke" prediction. We omit images that do not contain heavy smoke (smoke density 3) since heavy smoke is most important and easier to detect.
### Optimization
We use Adam optimizer, which uses both Momentum (0.99) and RMSProp methods. Our loss function is categorical cross-entropy over pixel-wise softmax. Let $\Omega$ be the dimension of output image and $\hat{\mathbf{x}}$ be the model output vector of pixel where $\hat{\mathbf{x}}\in\Omega$. $\hat{\mathbf{x}}_i$ represents the model output for $i$th category in $\hat{\mathbf{x}}$, where in our case $i\in\{0,1,2,3\}$. Then softmax is defined as 
$$p_k(\hat{\mathbf{x}})=\frac{\exp(\hat{\mathbf{x}}_k)}{\sum_{i=0}^3\exp(\hat{\mathbf{x}}_i)}.$$
Where $p_k(\hat{\mathbf{x}})$ represents the probability of pixel being category $k$ predicted by model. Let $q_k(\mathbf{x})$ be the correct probability of pixel $\mathbf{x}$ being category $k$ (it is either $0$ or $1$). Then the loss function is 
$$\mathcal{L}=\sum_{\mathbf{x}\in\Omega}\sum_{i\in\{0,1,2,3\}}q_i(\mathbf{x})\log(p_i(\hat{\mathbf{x}})).$$
It will penalize when the model outputs high probability for wrong class.

### Implementation
There are three files to do the training; `model.py`, `data.py`. We explain how they work below;
#### `model.py`
Our Unet model is written in Keras with tensorflow backend. To initiate a model instance, run

In [5]:
from model import unet
side_len = 256
model = unet(input_size=(side_len, side_len, 2) , pretrained_weights=None, learning_rate=1e-4)

Since you feed in square tiles of two bands of GOES into the model, the input shape has to be `(X, X, 2)` where `X` is tile's side length. To load a pre-trained weights, specify `pretrained_weights=path/to/weights`. You can also specify learning rate with `learning_rate`. Model implementation should be pretty straightforward.

#### `data.py`
`data.py` creates data loader for the unet model. To create a data loader instance, you should run something like this

In [6]:
from data import train_data_generator, stack_gen
batch_size = 32
seed = 5
validation_split_rate = 0.1
tile_dataframe = pd.read_csv('/n/mickley/users/ktoshima/csv/data_256.csv')
band1_gen, band3_gen, mask_gen, val_band1_gen, val_band3_gen, val_mask_gen = train_data_generator(
    dataframe=tile_dataframe,
    image_side_length=side_len,
    batch_size=batch_size,
    seed=seed,
    validation_split_rate=validation_split_rate
)

Found 6633514 non-validated image filenames.
Found 6633514 non-validated image filenames.
Found 6633514 non-validated image filenames.
Found 737057 non-validated image filenames.
Found 737057 non-validated image filenames.
Found 737057 non-validated image filenames.


`train_data_generator` takes in dataframe of tiles to be used in training, tile side length, validation split rate, batch size, and seed for reproducing randomization. It will return Keras' `ImageDataGenerator` of GOES band1, band3, and HMS for both training and validation. You feed data loader to model to train the model.

In [ ]:
# don't actually run this cell, because it will start training
epoch_num = 30
model.fit(
    x=stack_gen(band1_gen, band3_gen, mask_gen),
    epochs=epoch_num,
    verbose=1,
    steps_per_epoch=len(mask_gen),
    validation_data=stack_gen(val_band1_gen, val_band3_gen, val_mask_gen),
    validation_steps=len(val_mask_gen)
)

`stack_gen` will stack image generator and create a pair `((band1, band3), HMS)`.

With this, you can start your training, you still have to write an awful amount of code to do that. To assist you, we've prepared a helper Python script `main.py`.

### Start training with `main.py`
You can run your training by running below example;
```shell
python main.py --side-len=256 --batch-size=32 --smoke=dense --epoch-num=20
```

`main.py` takes in some arguments to configure your training.
- `--side-len` configures the side length of tiles. Has to be $2^n$.
- `--batch-size` configures batch size during traning. Has to be $2^n$.
- `--smoke` configures which tiles to use for training. If you use 'dense', you'll use tiles with at least 1 pixel of dense smoke pixel. If you use 'smoke', you'll use tiles with at least 1 pixel of smoke pixel. If you use 'all', you'll use all the tiles.
- `--epoch-num` configures the number of epochs you do the training.

For detailed usage, you can run
```shell
python main.py --help
```

Running `main.py` will create the following files;
- `train-{smoke}-by_date-{side_len}-{seed}.csv`: list of images used in training
- `test-{smoke}-by_date-{side_len}-{seed}.csv`: list of images to be used in testing
- `{smoke}-by_date-{side_len}-{seed}.hdf5`: weights of trained model
- `{smoke}-by_date-{side_len}-{seed}.csv`: history of training


## Strategy so far and moving forward
### Which tiles to be used
As mentioned, if we use all the tiles for training the model learns to produce nothing. So we've tried two ways of selecting tiles.
1. use tiles with at least 1 smoke pixel (smoke category non-zero)
2. use tiles with at least 1 dense smoke pixel (smoke category 3)

(1) did not work in that it resulted in almost the same result as using all the tiles. (2) worked slightly better but the model still produced almost no smoke for most of the prediction. Going forward, we might want to use tiles with certain amount of smoke. Fortunately, when we produce tiles we measure the sum of HMS pixel value at `hms_sum` column for each tile. As a future work, we can consider finding a certain threshold for `hms_sum` to be used in training.

### Pick by date
One of our key findings about HMS is that it is likely to be produced twice a day. This means that most of HMS images within a certain day are almost identical. If so, when we split up the tiles into training and test dataset we would end up having a lot of "overlap" of images between training and test dataset. This could result in overfitting. To resolve this issue, we introduced "pick by date" method, which is that we split up training and test dataset by splitting up dates into two groups and create training and test dataset by picking up tiles of those dates. This is implemented in `main.py`, so see the file for implementation.
This "pick by date" method worsened the model performance.

In [7]:
# you can see hms_sum column
tile_dataframe.head()

,timestamp,num,path_band1,path_band3,path_hms,hms_sum,is_dense,not_void,not_edge,not_night,row_17,col_39,valid
0,2017-07-11 00:00:00,0,/n/holyscratch01/mickley/hms_vision_data/256x2...,/n/holyscratch01/mickley/hms_vision_data/256x2...,/n/holyscratch01/mickley/hms_vision_data/256x2...,0,False,False,True,True,0,0,False
1,2017-07-11 00:00:00,1,/n/holyscratch01/mickley/hms_vision_data/256x2...,/n/holyscratch01/mickley/hms_vision_data/256x2...,/n/holyscratch01/mickley/hms_vision_data/256x2...,0,False,False,True,True,0,1,False
2,2017-07-11 00:00:00,2,/n/holyscratch01/mickley/hms_vision_data/256x2...,/n/holyscratch01/mickley/hms_vision_data/256x2...,/n/holyscratch01/mickley/hms_vision_data/256x2...,0,False,False,True,True,0,2,False
3,2017-07-11 00:00:00,3,/n/holyscratch01/mickley/hms_vision_data/256x2...,/n/holyscratch01/mickley/hms_vision_data/256x2...,/n/holyscratch01/mickley/hms_vision_data/256x2...,0,False,False,True,True,0,3,False
4,2017-07-11 00:00:00,4,/n/holyscratch01/mickley/hms_vision_data/256x2...,/n/holyscratch01/mickley/hms_vision_data/256x2...,/n/holyscratch01/mickley/hms_vision_data/256x2...,0,False,False,True,True,0,4,False
